# 如何从工具流式传输事件
:::info 前提条件
本指南假定您已熟悉以下概念：- [LangChain 工具](/docs/concepts/tools)- [自定义工具](/docs/how_to/custom_tools)- [使用流事件](/docs/how_to/streaming/#使用流事件)- [在自定义工具中访问 RunnableConfig](/docs/how_to/tool_configure/)
:::
如果你有调用[聊天模型](/docs/concepts/chat_models/)、[检索器](/docs/concepts/retrievers/)或其他[可运行项](/docs/concepts/runnables/)的[工具](/docs/concepts/tools/)，你可能需要访问这些可运行项的内部事件，或用额外属性配置它们。本指南将展示如何正确手动传递参数，以便你能通过`astream_events()`方法实现这一需求。
:::caution 兼容性
LangChain 无法自动将配置（包括 `astream_events()` 所需的回调）传播到子可运行对象，如果您在 `python<=3.10` 中运行 `async` 代码。这是您可能无法从自定义可运行对象或工具中看到事件发出的常见原因。
如果您正在运行 Python≤3.10 版本，在异步环境中需要手动将 `RunnableConfig` 对象传递给子可运行对象。关于如何手动传递配置的示例，请参阅下方 `bar` RunnableLambda 的实现。
如果您正在运行 Python 3.11 或更高版本，`RunnableConfig` 在异步环境中会自动传播到子可运行对象。不过，如果您的代码可能在较旧版本的 Python 中运行，手动传播 `RunnableConfig` 仍然是一个好习惯。
本指南还要求 `langchain-core>=0.2.16`。:::
假设你有一个自定义工具，它调用一个链式流程来压缩输入内容：先提示聊天模型仅返回10个单词，然后将输出结果反转。首先，用最基础的方式定义它：
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs customVarName="model" />


In [1]:
# | output: false
# | echo: false

import os
from getpass import getpass

from langchain_anthropic import ChatAnthropic

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass()

model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool


@tool
async def special_summarization_tool(long_text: str) -> str:
    """A tool that summarizes input text using advanced techniques."""
    prompt = ChatPromptTemplate.from_template(
        "You are an expert writer. Summarize the following text in 10 words or less:\n\n{long_text}"
    )

    def reverse(x: str):
        return x[::-1]

    chain = prompt | model | StrOutputParser() | reverse
    summary = await chain.ainvoke({"long_text": long_text})
    return summary

直接调用该工具完全可行：

In [3]:
LONG_TEXT = """
NARRATOR:
(Black screen with text; The sound of buzzing bees can be heard)
According to all known laws of aviation, there is no way a bee should be able to fly. Its wings are too small to get its fat little body off the ground. The bee, of course, flies anyway because bees don't care what humans think is impossible.
BARRY BENSON:
(Barry is picking out a shirt)
Yellow, black. Yellow, black. Yellow, black. Yellow, black. Ooh, black and yellow! Let's shake it up a little.
JANET BENSON:
Barry! Breakfast is ready!
BARRY:
Coming! Hang on a second.
"""

await special_summarization_tool.ainvoke({"long_text": LONG_TEXT})

'.yad noitaudarg rof tiftuo sesoohc yrraB ;scisyhp seifed eeB'

但如果你想访问聊天模型的原始输出而非完整工具，可以尝试使用 [`astream_events()`](/docs/how_to/streaming/#using-stream-events) 方法并监听 `on_chat_model_end` 事件。具体过程如下：

In [4]:
stream = special_summarization_tool.astream_events({"long_text": LONG_TEXT})

async for event in stream:
    if event["event"] == "on_chat_model_end":
        # Never triggers in python<=3.10!
        print(event)

你会注意到（除非你是在 `python>=3.11` 环境下运行本指南），子运行中并未触发任何聊天模型事件！
这是因为上述示例未将工具的配置对象传递至内部链中。要解决此问题，需重新定义工具以接收一个特殊参数（类型为`RunnableConfig`，详见[本指南](/docs/how_to/tool_configure)）。在执行内部链时，还需将该参数透传下去：

In [5]:
from langchain_core.runnables import RunnableConfig


@tool
async def special_summarization_tool_with_config(
    long_text: str, config: RunnableConfig
) -> str:
    """A tool that summarizes input text using advanced techniques."""
    prompt = ChatPromptTemplate.from_template(
        "You are an expert writer. Summarize the following text in 10 words or less:\n\n{long_text}"
    )

    def reverse(x: str):
        return x[::-1]

    chain = prompt | model | StrOutputParser() | reverse
    # Pass the "config" object as an argument to any executed runnables
    summary = await chain.ainvoke({"long_text": long_text}, config=config)
    return summary

现在尝试用你的新工具执行与之前相同的 `astream_events()` 调用：

In [6]:
stream = special_summarization_tool_with_config.astream_events({"long_text": LONG_TEXT})

async for event in stream:
    if event["event"] == "on_chat_model_end":
        print(event)

{'event': 'on_chat_model_end', 'data': {'output': AIMessage(content='Bee defies physics; Barry chooses outfit for graduation day.', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-337ac14e-8da8-4c6d-a69f-1573f93b651e', usage_metadata={'input_tokens': 182, 'output_tokens': 19, 'total_tokens': 201, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}), 'input': {'messages': [[HumanMessage(content="You are an expert writer. Summarize the following text in 10 words or less:\n\n\nNARRATOR:\n(Black screen with text; The sound of buzzing bees can be heard)\nAccording to all known laws of aviation, there is no way a bee should be able to fly. Its wings are too small to get its fat little body off the ground. The bee, of course, flies anyway because bees don't care what humans think is impossible.\nBARRY BENSON:\n(Barry is picking out a shirt)\nYellow, black. Yellow, black. Yellow, black. Yellow, black. Ooh, black and yellow! Let's s

太棒了！这次有一个事件被触发了。
对于流式传输，`astream_events()`会在可能的情况下自动启用流式功能调用链中的内部可运行对象。因此，如果您想要获取聊天模型生成时的令牌流，只需筛选查找`on_chat_model_stream`事件即可，无需其他改动：

In [7]:
stream = special_summarization_tool_with_config.astream_events({"long_text": LONG_TEXT})

async for event in stream:
    if event["event"] == "on_chat_model_stream":
        print(event)

{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-f5e049f7-4e98-4236-87ab-8cd1ce85a2d5', usage_metadata={'input_tokens': 182, 'output_tokens': 2, 'total_tokens': 184, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})}, 'run_id': 'f5e049f7-4e98-4236-87ab-8cd1ce85a2d5', 'name': 'ChatAnthropic', 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-5-sonnet-20240620', 'ls_model_type': 'chat', 'ls_temperature': 0.0, 'ls_max_tokens': 1024}, 'parent_ids': ['51858043-b301-4b76-8abb-56218e405283']}
{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='Bee', additional_kwargs={}, response_metadata={}, id='run-f5e049f7-4e98-4236-87ab-8cd1ce85a2d5')}, 'run_id': 'f5e049f7-4e98-4236-87ab-8cd1ce85a2d5', 'name': 'ChatAnthropic', 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-5-sonnet-20240620', 'ls_model

## 后续步骤
你现在已经了解了如何从工具内部流式传输事件。接下来，请查看以下指南以了解更多关于工具使用的信息：
- 将[运行时值传递给工具](/docs/how_to/tool_runtime)- 将[工具结果传回模型](/docs/how_to/tool_results_pass_to_model)- [调度自定义回调事件](/docs/how_to/callbacks_custom_events)
您还可以查看一些更具体的工具调用用例：
- 构建 [工具使用链和智能体](/docs/how_to#tools)- 从模型获取[结构化输出](/docs/how_to/structured_output/)